In [1]:
import sys
sys.path.append(".venv/lib/python3.11/site-packages")

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

In [20]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):

        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
from torch import tensor, backends
backends.mps.is_available()

True

In [17]:
device = torch.device('mps')
model.to("mps")
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 2.184990  [   64/60000]
loss: 2.170162  [ 6464/60000]
loss: 2.114387  [12864/60000]
loss: 2.124988  [19264/60000]
loss: 2.085786  [25664/60000]
loss: 2.022904  [32064/60000]
loss: 2.051327  [38464/60000]
loss: 1.984763  [44864/60000]
loss: 1.987278  [51264/60000]
loss: 1.916833  [57664/60000]
Test Error: 
 Accuracy: 60.7%, Avg loss: 1.913270 

Epoch 2
-------------------------------
loss: 1.947420  [   64/60000]
loss: 1.914623  [ 6464/60000]
loss: 1.799428  [12864/60000]
loss: 1.838351  [19264/60000]
loss: 1.736452  [25664/60000]
loss: 1.676430  [32064/60000]
loss: 1.698398  [38464/60000]
loss: 1.605745  [44864/60000]
loss: 1.622954  [51264/60000]
loss: 1.517915  [57664/60000]
Test Error: 
 Accuracy: 62.9%, Avg loss: 1.534288 

Epoch 3
-------------------------------
loss: 1.601139  [   64/60000]
loss: 1.562105  [ 6464/60000]
loss: 1.410374  [12864/60000]
loss: 1.484658  [19264/60000]
loss: 1.364230  [25664/60000]
loss: 1.351771  [32064/600

In [21]:
device = torch.device('cpu')
model.to("cpu")
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295559  [   64/60000]
loss: 2.288636  [ 6464/60000]
loss: 2.273428  [12864/60000]
loss: 2.267638  [19264/60000]
loss: 2.253835  [25664/60000]
loss: 2.223847  [32064/60000]
loss: 2.224128  [38464/60000]
loss: 2.198083  [44864/60000]
loss: 2.195476  [51264/60000]
loss: 2.150772  [57664/60000]
Test Error: 
 Accuracy: 48.7%, Avg loss: 2.157439 

Epoch 2
-------------------------------
loss: 2.166163  [   64/60000]
loss: 2.158852  [ 6464/60000]
loss: 2.108067  [12864/60000]
loss: 2.119490  [19264/60000]
loss: 2.079686  [25664/60000]
loss: 2.014477  [32064/60000]
loss: 2.034462  [38464/60000]
loss: 1.962724  [44864/60000]
loss: 1.971719  [51264/60000]
loss: 1.884464  [57664/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.897514 

Epoch 3
-------------------------------
loss: 1.930746  [   64/60000]
loss: 1.900364  [ 6464/60000]
loss: 1.792769  [12864/60000]
loss: 1.827678  [19264/60000]
loss: 1.728536  [25664/60000]
loss: 1.673933  [32064/600

In [ ]:
torch.save(model, 'model.pth')
model = torch.load('model.pth')